In [13]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import numpy as np
from sklearn.metrics import mean_squared_error

In [14]:
data = pd.read_csv(r'flats_to_buy_preproceesed.csv')


data.dropna(subset=['ConstructionYear'], inplace=True)
data.dropna(subset=['Object_price'], inplace=True)
data.dropna(subset=['Rooms'], inplace=True)
data.drop('Title', axis=1, inplace=True)
data.drop('Object_currency', axis=1, inplace=True)
data.drop('Url', axis=1, inplace=True)
data.drop('EstateType', axis=1, inplace=True)
data.drop('DistributionType', axis=1, inplace=True)
data = data.reindex()
data = data.reset_index(drop=True)

y = data['Object_price']
X = data.drop('Object_price', axis=1)
print(y.head())
print(X.head())

print(data['als_ferienimmobilie_geeignet'])

0    220000.0
1    269000.0
2    169000.0
3    275000.0
4    239000.0
Name: Object_price, dtype: float64
   LivingSpace  Rooms  ConstructionYear  ZipCode EstateTypeGerman  \
0        53.00    2.0            1971.0    97080          WOHNUNG   
1        70.00    3.0            1968.0    97078          WOHNUNG   
2        33.51    1.0            1983.0    97072          WOHNUNG   
3        65.45    2.0            1958.0    97072          WOHNUNG   
4        79.50    3.0            1977.0    97209          WOHNUNG   

  DistributionTypeGerman  abstellraum  als_ferienimmobilie_geeignet  \
0               ZUM_KAUF            0                             0   
1               ZUM_KAUF            0                             0   
2               ZUM_KAUF            0                             0   
3               ZUM_KAUF            0                             0   
4               ZUM_KAUF            1                             0   

   altbau_(bis_1945)  bad/wc_getrennt  ...  teppich  

In [15]:
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                  train_size=0.8, 
                                                  random_state = 0)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, 
                                                  train_size=0.8, 
                                                  random_state = 0)

num_cols = [col for col in train_X.columns if train_X[col].dtype == 'float64']
cat_cols = [col for col in train_X.columns if train_X[col].dtype == 'object']
bin_cols = [col for col in train_X.columns if train_X[col].dtype == 'int64']

In [16]:
print(bin_cols)

['ZipCode', 'abstellraum', 'als_ferienimmobilie_geeignet', 'altbau_(bis_1945)', 'bad/wc_getrennt', 'balkon', 'barriefrei', 'block', 'carport', 'dachgeschoss', 'denkmalgeschuetzt', 'dielen', 'dsl', 'duplex', 'dusche', 'einbauk\\u00fcche', 'elektro', 'erdgeschoss', 'erstbezug', 'estrich', 'etagenheizung', 'fenster', 'fern', 'ferne', 'fliesen', 'fluessiggas', 'frei', 'fu\\u00dfbodenheizung', 'gaestewc', 'garage', 'garten', 'gartennutzung', 'gas', 'gepflegt', 'haustiere_erlaubt', 'holzfenster', 'kable_sat_tv', 'kamera', 'kamin', 'kapitalanlage', 'kelleranteil', 'kfw40', 'kfw55', 'kfw70', 'kontrollierte_be-_und_entl\\u00fcftungsanlage', 'kunststoff', 'kunststofffenster', 'laminat', 'linoleum', 'loggia', 'luftwp', 'massivhaus', 'moebliert', 'neubau', 'neuwertig', 'oel', 'offene_k\\u00fcche', 'pantry', 'parkett', 'pellet', 'personenaufzug', 'provisionsfrei', 'reinigung', 'renoviert', 'renovierungsbed\\u00fcrftig', 'rollstuhlgerecht', 'sat', 'sauna', 'souterrain', 'speisekammer', 'stein', 'ste

In [17]:
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_xgb = encoder.fit_transform(train_X[cat_cols]).toarray()
X_valid_categorical_xgb = encoder.transform(val_X[cat_cols]).toarray()
X_test_categorical_xgb = encoder.transform(test_X[cat_cols]).toarray()
X_train_processed = np.concatenate([train_X[num_cols], X_train_categorical_xgb], axis=1)
X_valid_processed = np.concatenate([val_X[num_cols], X_valid_categorical_xgb], axis=1)
X_test_processed = np.concatenate([test_X[num_cols], X_test_categorical_xgb], axis=1)
X_train_processed = np.concatenate([train_X[bin_cols], X_train_processed], axis=1)
X_valid_processed = np.concatenate([val_X[bin_cols], X_valid_processed], axis=1)
X_test_processed = np.concatenate([test_X[bin_cols], X_test_processed], axis=1)

In [18]:
X_train_processed[0]

array([9.7076e+04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e

In [25]:
mlflow.end_run()

In [26]:
mlflow.set_tracking_uri("http://localhost:5000")


In [27]:
import mlflow
import mlflow.xgboost
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Start an MLflow run
mlflow.start_run()

# Log the parameters
mlflow.log_params({
    "eval_metric": ['rmse', 'mae'],
    "early_stopping_rounds": 15,

})

# Train the XGBoost model
model = xgb.XGBRegressor(eval_metric=['rmse', 'mae'], early_stopping_rounds=15)
model.fit(X=X_train_processed, 
          y=train_y,
          eval_set=[(X_valid_processed, val_y)],
          verbose=True)


# Log the model
mlflow.xgboost.log_model(model, "xgboost-model")

# Make predictions on the test set
preds = model.predict(X_test_processed)
score = mean_absolute_error(test_y, preds)


# End the MLflow run
mlflow.end_run()


[0]	validation_0-rmse:377643.68046	validation_0-mae:262610.76982
[1]	validation_0-rmse:315303.28482	validation_0-mae:198007.32736
[2]	validation_0-rmse:273814.66459	validation_0-mae:156059.12976
[3]	validation_0-rmse:247366.68043	validation_0-mae:130843.66825
[4]	validation_0-rmse:227769.44017	validation_0-mae:111355.84699
[5]	validation_0-rmse:218264.07826	validation_0-mae:101177.79649
[6]	validation_0-rmse:213008.86382	validation_0-mae:94187.91902
[7]	validation_0-rmse:208510.08491	validation_0-mae:88551.81269
[8]	validation_0-rmse:205527.83342	validation_0-mae:85034.76410
[9]	validation_0-rmse:199592.69830	validation_0-mae:82180.19207
[10]	validation_0-rmse:197761.46761	validation_0-mae:80659.57965
[11]	validation_0-rmse:195102.59977	validation_0-mae:79670.42111
[12]	validation_0-rmse:193166.98161	validation_0-mae:79049.88700
[13]	validation_0-rmse:190970.01320	validation_0-mae:78111.36986
[14]	validation_0-rmse:190713.60367	validation_0-mae:78035.53316
[15]	validation_0-rmse:189936